In [ ]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit_aer import AerSimulator

from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

service = QiskitRuntimeService(channel="ibm_quantum")

In [ ]:
# the bases we need to measure the state in, 81=3^4 in total
# the bases go from XXXX to ZZZZ in lexicographical order
bases = []
for i in range(3):
    for j in range(3):
        for k in range(3):
            for l in range(3):
                bases.append([i, j, k, l])

In [ ]:
# layouts that achieve a 9 CNOT circuit on the 7 qubit topology
layouts_7qubit = [
    [1,3,0,5], 
    [3,5,1,4],
    [1,3,2,5],
    [3,5,1,6],
    [3,1,5,0],
    [5,3,4,1],
    [3,1,5,2],
    [5,3,6,1]
          ]

In [ ]:
# layouts that achieve a 9 CNOT circuit on the 5 qubit linear topology
layouts_5qubit_linear = [
    [1,2,0,3],
    [2,1,3,0],
    [2,3,1,4],
    [3,2,4,1]
]

In [ ]:
# layouts that achieve a 9 CNOT circuit on the 5 qubit T-shape topology
layouts_5qubit_Tshape = [
    [1,3,0,4],
    [3,1,4,0],
    [1,3,2,4],
    [3,1,4,2]
]

In [ ]:
def create_tomo_circuit(basis, initial_layout, backend):
    '''
    create the quantum circuit of the cabello state and measure in the chosen basis
    the basis is a 4-element vector containing 0s, 1s and 2s 
    from [0, 0, 0, 0] meaning XXXX to [2, 2, 2, 2] meaning ZZZZ
    parameters:
    ----------
    basis: the basis is a 4-element vector containing 0s, 1s and 2s 
        from [0, 0, 0, 0] meaning XXXX to [2, 2, 2, 2] meaning ZZZZ
    initial_layout: which qubits of the backend to use
    backend: which backend to use
    '''
    # the cabello circuit creating the state 2|1100>+2|0011>-|1010>-|0101>-|1001>-|0110>
    istvan_circ = QuantumCircuit(4, 4)
    istvan_circ.h(0)
    istvan_circ.h(1)
    istvan_circ.h(2)
    istvan_circ.rz(-0.73304, 0)
    istvan_circ.rz(2.67908, 2)
    istvan_circ.cnot(2, 0)
    istvan_circ.ry(-2.67908, 0)
    istvan_circ.h(2)
    istvan_circ.cnot(1, 0)
    istvan_circ.cnot(2, 3)
    istvan_circ.rz(1.5708, 2)
    istvan_circ.cnot(1, 3)
    istvan_circ.cnot(0, 2)
    # measurement. result '0' in the X basis means state |+>, in the Y basis |+i> and in the Z basis |0>
    for i in range(4):
        if basis[i] == 0:
            istvan_circ.h(i)
            istvan_circ.measure(i, i)
        elif basis[i] == 1:
            istvan_circ.sdg(i)
            istvan_circ.h(i)
            istvan_circ.measure(i, i)
        else:
            istvan_circ.measure(i, i)
    return transpile(istvan_circ, backend=backend, initial_layout=initial_layout, optimization_level=3)

In [ ]:
def error_circuits(layout, backend):
    '''create the quantum circuit for tensor product readout error measurement
    parameters: 
    ----------
    layout: which qubits of the computer to measure
    n: number of qubits in the circuit'''
    circs = []
    n = backend.configuration().n_qubits
    for qubit in layout:
        circs.append(QuantumCircuit(n, n))
        circs[-1].x(qubit)
        circs[-1].measure(range(n), range(n))
    return circs

In [ ]:
backend = service.get_backend('ibmq_quito')

In [ ]:
def run_circuits(backend)
    '''runs the circuits on all layouts that achieve a 9 CNOT circuit'''
    all_res = []
    n = backend.configuration().n_qubits
    if n == 7:
        layouts = layouts_7qubit
    if n == 5:
        if backend.name == 'ibmq_manila':
            layouts = layouts_5qubit_linear
        else:
            layouts = layouts_5qubit_Tshape
    for ind, layout in enumerate(layouts):
        sampler = Sampler(backend)
        circs = []
        for basis in bases:
            circs.append(create_tomo_circuit(basis, layout, backend))
        circs.extend(error_circuits(layout, backend))
        res = sampler.run(circs, resilience_level=0, optimization_level=0)
        all_res.append(res)